In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\AI\\Deep Learning\\Generative AI\\My Projects\\8-End to End Medical Chatbot\\End-to-End-Medical-Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\AI\\Deep Learning\\Generative AI\\My Projects\\8-End to End Medical Chatbot\\End-to-End-Medical-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
# extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [11]:
# text_chunks

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os

In [16]:
load_dotenv()

True

In [17]:
def get_gemini_embeddings():
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001", 
        google_api_key=os.getenv("GOOGLE_API_KEY") 
    )
    return embeddings

In [18]:
embeddings = get_gemini_embeddings()

In [63]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 768


In [19]:
# query_result

In [22]:
import os
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [24]:
from langchain_community.vectorstores import FAISS

In [29]:
# 1. Create FAISS index from documents
docsearch = FAISS.from_documents(
    documents=text_chunks,  
    embedding=embeddings
)

In [30]:
# 2: Save the FAISS index locally (optional)
docsearch.save_local("faiss_medicalbot_index")

In [51]:
# 3: Load the FAISS index later (if needed)
docsearch = FAISS.load_local(
    folder_path="faiss_medicalbot_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True  # Required for FAISS
)

In [52]:
docsearch

In [53]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [54]:
retrieved_docs = retriever.invoke("What is Acne?")

In [55]:
retrieved_docs

[Document(id='1c5b2a3e-c22d-4be3-b680-e0f827b04f11', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 38, 'page_label': '39'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='abd652e5-ee07-4c4f-9fe8-046c61745784', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 239, 'page_label': '240'}, page_content='Isotretinoin (Accutane) is prescribed only for very\

In [56]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.4,    
    max_output_tokens=500,  
    google_api_key=os.getenv("GOOGLE_API_KEY")  
)

In [58]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [59]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [64]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Acne is a skin condition where raised bumps, pimples, and cysts form on the face, neck, shoulders, and upper back. It occurs when pores or hair follicles become blocked, causing sebum to collect inside. Bacteria and dead skin cells can also collect, leading to inflammation.


In [61]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I am sorry, but the provided text does not contain information about the definition of "stats".
